# Tutorial OpenCL 

Requisitos:
- OpenCL 1.2
- Python 2.7.16 (pyopencl 2019.1.1, numpy 1.16.5, pandas 0.24.2)

## Preparo do Notebook 
Antes de começar a usar este notebook, execute a célula abaixo !

In [ ]:
!pip3 install pyopencl
!sudo apt install subversion

#Download files from Github 
!svn checkout https://github.com/menotti/pp/trunk/code/OpenCL/ 
!mv OpenCL/*.cl ./ && rm -fr OpenCL

In [ ]:
#
# Matrix Multiplication Driver
#
# This is a driver program to test various ways of computing
# the product:
#                 C = A * B
#
# A and B are constant matrices, square and the order is
# set as a constant, ORDER (see definitions.py). This is so
# we can make a quick test of the multiplication result.
#
# History:   C++ version written by Tim Mattson, August 2010 
#            Modified by Simon McIntosh-Smith, September 2011
#            Modified by Tom Deakin and Simon McIntosh-Smith, October 2012
#            Ported to Python by Tom Deakin, July 2013
#            Ported to Jupyter Notebooks by Ricardo Menotti, May 2020

from time import time
import pyopencl as cl
import pandas as pd
import numpy
import os

# Order of the square matrices A, B and C
ORDER = 1024

# A elemetns are constant and equal to AVAL
AVAL = 3.0

# B elemetns are constant and equal to BVAL
BVAL = 5.0

# tolerance used in floating point comparisons
TOL = 0.001

# Max dim for NDRange
DIM = 2

# File to read/write results
FILENAME = 'results.csv'

# number of times to do each multiplication
COUNT = 10

# Creating Pandas DataFrame 
if os.path.isfile(FILENAME):
    df = pd.read_csv(FILENAME)
else:
    df = pd.DataFrame({"version": [], "execution_time": []})

#  Function to compute the matrix product (sequential algorithm, dot prod)
def seq_mat_mul_sdot(N, A, B, C):
    for i in range(N):
        for j in range(N):
            tmp = 0.0
            for k in range(N):
                tmp += A[i*N+k] * B[k*N+j]
            C[i*N+j] = tmp

#  Function to compute errors of the product matrix
def error(N, C):
   cval = float(N) * AVAL * BVAL
   errsq = 0.0
   for i in range(N):
       for j in range(N):
            err = C[i*N+j] - cval
            errsq += err * err
   return errsq;

# Function to analyze and output results
def results(N, C, run_time):
    mflops = 2.0 * N * N * N/(1000000.0* run_time)
    print( run_time, "seconds at", mflops, "MFLOPS")
    errsq = error(N, C)
    if (errsq > TOL):
        print("Errors in multiplication:", errsq)

# A[N][N], B[N][N], C[N][N]
N = ORDER;

# Number of elements in the matrix
size = N * N

# A matrix
h_A = numpy.empty(size).astype(numpy.float32)
h_A.fill(AVAL)

# B matrix
h_B = numpy.empty(size).astype(numpy.float32)
h_B.fill(BVAL)

# C matrix
h_C = numpy.empty(size).astype(numpy.float32)

Em Python o OpenCL é um pouco mais sucinto, por exemplo, basta as duas linhas abaixo para criar um fila associada a um dispositivo/contexto. 

In [ ]:
# Set the environment variable to choose platform/device 
# os.environ["PYOPENCL_CTX"] = '0:0'

# Setup OpenCL
platform = cl.get_platforms()
my_gpu_devices = platform[0].get_devices(device_type=cl.device_type.GPU)
context = cl.Context(devices=my_gpu_devices)
# context = cl.create_some_context()
queue = cl.CommandQueue(context)

# Reset host buffers - just to play it safe
h_A = numpy.empty(size).astype(numpy.float32)
h_A.fill(AVAL)
h_B = numpy.empty(size).astype(numpy.float32)
h_B.fill(BVAL)
h_C = numpy.empty(size).astype(numpy.float32)

# Create OpenCL buffers
d_a = cl.Buffer(context, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR, hostbuf=h_A)
d_b = cl.Buffer(context, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR, hostbuf=h_B)
d_c = cl.Buffer(context, cl.mem_flags.WRITE_ONLY, h_C.nbytes)

Na versão mais simples, cada _work-item_ calcula um elemento da matriz:

In [ ]:
!cat C_elem.cl

In [ ]:
#--------------------------------------------------------------------------------
# OpenCL matrix multiplication ... Naive
#--------------------------------------------------------------------------------

kernelsource = open("C_elem.cl").read()
program = cl.Program(context, kernelsource).build()
mmul = program.mmul
mmul.set_scalar_arg_dtypes([numpy.int32, None, None, None])
print( "\n===== OpenCL, matrix mult, C(i,j) per work item, order", N, "======\n")

# Do the multiplication COUNT times
for i in range(COUNT):
    h_C.fill(0.0)
    start_time = time()
    mmul(queue, (N, N), None, N, d_a, d_b, d_c)
    queue.finish()
    run_time = time() - start_time
    cl.enqueue_copy(queue, h_C, d_c)
    results(N, h_C, run_time)
    df = df.append(pd.DataFrame({"version": ["naive"], 
                                 "execution_time": [run_time]}))
df.to_csv(FILENAME, index = False, header = True)

Nesta versão, cada _work-item_ calcula uma linha da matriz:

In [ ]:
!cat C_row.cl

In [ ]:
#--------------------------------------------------------------------------------
# OpenCL matrix multiplication ... C row per work item
#--------------------------------------------------------------------------------

kernelsource = open("C_row.cl").read()
program = cl.Program(context, kernelsource).build()
mmul = program.mmul
mmul.set_scalar_arg_dtypes([numpy.int32, None, None, None])
print( "\n===== OpenCL, matrix mult, C row per work item, order", N, "======\n")
# Do the multiplication COUNT times
for i in range(COUNT):
    h_C.fill(0.0)
    start_time = time()
    mmul(queue, (N,), (ORDER//16,), N, d_a, d_b, d_c)
    queue.finish()
    run_time = time() - start_time
    cl.enqueue_copy(queue, h_C, d_c)
    results(N, h_C, run_time)
    df = df.append(pd.DataFrame({"version": ["C row per work item"], 
                                 "execution_time": [run_time]}))
df.to_csv(FILENAME, index = False, header = True)

Agora fazemos o mesmo, mas antes copiamos uma linha da matriz A em memória privada 

In [ ]:
!cat C_row_priv.cl

In [ ]:
#--------------------------------------------------------------------------------
# OpenCL matrix multiplication ... C row per work item, A row in private memory
#--------------------------------------------------------------------------------
PYBIND11_DETAILED_ERROR_MESSAGES = 1
kernelsource = open("C_row_priv.cl").read()
program = cl.Program(context, kernelsource).build()
mmul = program.mmul
mmul.set_scalar_arg_dtypes([numpy.int32, None, None, None])
print( "\n===== OpenCL, matrix mult, C row, A row in priv mem, order", N, "======\n")
# Do the multiplication COUNT times
for i in range(COUNT):
    h_C.fill(0.0)
    start_time = time()
    mmul(queue, (N,), (ORDER//16,), N, d_a, d_b, d_c)
    queue.finish()
    run_time = time() - start_time
    cl.enqueue_copy(queue, h_C, d_c)
    results(N, h_C, run_time)
    df = df.append(pd.DataFrame({"version": ["C row, A row in priv mem"], 
                                 "execution_time": [run_time]}))
df.to_csv(FILENAME, index = False, header = True)

Agora vamos usar a memória local para a coluna B da matriz a ser calculada 

In [ ]:
!cat C_row_priv_bloc.cl

In [ ]:
#--------------------------------------------------------------------------------
# OpenCL matrix multiplication ... C row per work item, A row private, B col local
#--------------------------------------------------------------------------------

kernelsource = open("C_row_priv_bloc.cl").read()
program = cl.Program(context, kernelsource).build()
mmul = program.mmul
mmul.set_scalar_arg_dtypes([numpy.int32, None, None, None, None])
print ("\n===== OpenCL, mat mult, C row, priv A, B cols loc, order", N, "======\n")
# Do the multiplication COUNT times
for i in range(COUNT):
    h_C.fill(0.0)
    start_time = time()
    localmem = cl.LocalMemory(numpy.dtype(numpy.float32).itemsize * N)
    mmul(queue, (N,), (ORDER//16,), N, d_a, d_b, d_c, localmem)
    queue.finish()
    run_time = time() - start_time
    cl.enqueue_copy(queue, h_C, d_c)
    results(N, h_C, run_time)
    df = df.append(pd.DataFrame({"version": ["C row, priv A, B cols loc"], 
                                 "execution_time": [run_time]}))
df.to_csv(FILENAME, index = False, header = True)

Por fim, temos uma versão que calcula por blocos:

In [ ]:
!cat C_block_form.cl

In [ ]:
#--------------------------------------------------------------------------------
# OpenCL matrix multiplication ... blocked
#--------------------------------------------------------------------------------

kernelsource = open("C_block_form.cl").read()
program = cl.Program(context, kernelsource).build()
mmul = program.mmul
mmul.set_scalar_arg_dtypes([numpy.int32, None, None, None, None, None])
print("\n==== Parallel matrix mult (blocked), order {0} on device ======\n".format(N))
# Do the multiplication COUNT times
for i in range(COUNT):
    h_C.fill(0.0)
    start_time = time()
    # Work-group computes a block of C. This size is also set
    # in a #define inside the kernel function. Note this blocksize
    # must evenly divide the matrix order
    blocksize = 16
    A_block = cl.LocalMemory(numpy.dtype(numpy.float32).itemsize * blocksize * blocksize)
    B_block = cl.LocalMemory(numpy.dtype(numpy.float32).itemsize * blocksize * blocksize)
    mmul(queue, (N,N), (blocksize,blocksize), N,
        d_a, d_b, d_c, A_block, B_block)
    queue.finish()
    run_time = time() - start_time
    cl.enqueue_copy(queue, h_C, d_c)
    results(N, h_C, run_time)
    df = df.append(pd.DataFrame({"version": ["blocked"], 
                                 "execution_time": [run_time]}))
df.to_csv(FILENAME, index = False, header = True)

In [ ]:
# Plot results
df = pd.read_csv(FILENAME)
by_version = df.groupby(by="version", sort=False)
avg_time = by_version.mean()
std = by_version.std()
plt = avg_time.plot(legend=False, kind="bar", yerr=std);
plt.set_title("Average execution time");
plt.set_ylabel("Execution time (seconds)");
plt.get_figure().savefig("results.pdf")

# Referências

* [Hands On OpenCL](http://handsonopencl.github.io/)